In [574]:
import pandas as pd
from nltk.stem import PorterStemmer
import os
import json
from datetime import datetime
from datetime import timedelta
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

In [575]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import nltk.corpus  
from nltk.text import Text
import pandas as pd
import re
import nltk
#from nltk.corpus import gutenber

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [576]:
!pip install transformers
import torch
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

OSError: [Errno 12] Cannot allocate memory

### Load data

In [85]:
clean_pdf_df = pd.read_csv("/home/jupyter/covid19-challenge/data/clean_doc_pdf.csv")

In [86]:
metadata = pd.read_csv("/home/jupyter/covid19-challenge/data/metadata.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [87]:
metadata.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [321]:
clean_pdf_df.columns

Index(['paper_id', 'title_x', 'authors_x', 'affiliations', 'abstract_x',
       'text', 'bibliography', 'raw_authors', 'raw_bibliography', 'sha',
       'title_y', 'authors_y', 'abstract_y', 'doi', 'publish_time', 'journal',
       'text_cleaned'],
      dtype='object')

In [326]:
#https://www.kaggle.com/colinlagator/cord-19-regex-and-webscraping-booz-allen
keyword_list = ['novel coronavirus', 'novel-coronavirus', 'coronavirus-2019', 'sars-cov-2', 'sarscov2', 'covid-19', 'covid19',
                        '2019ncov', '2019-ncov', 'wuhan']


covid19_index_list = []
for i, row in clean_pdf_df.iterrows():
    text = row['text'].lower()
    has_keyword = False
    for keyword in keyword_list:
        if keyword in text:
            has_keyword = True
            break
    if has_keyword == True:
        covid19_index_list.append(i)

19

In [ ]:
clean_pdf_df = clean_pdf_df.iloc[covid19_index_list, :]

In [88]:
#len(set(clean_pdf_df['paper_id']).intersection(metadata['sha']))  # ~7,000 papers not in common

In [89]:
clean_pdf_df = clean_pdf_df.merge(metadata[['sha', 'title', 'authors', 'abstract', 'doi', 'publish_time', 'journal']], how ='left', left_on='paper_id', right_on='sha')

### Define functions

#### Data preprocessing

In [90]:
porter_stemmer = PorterStemmer()
def clean_sent(sentence):
    """
    Clean the sentence
    :param sentence: text to to be cleaned
    :return: text that has been cleaned
    """
    #nltk.FreqDist(words).most_common(10)
    stopwords = set(nltk.corpus.stopwords.words('english'))
    words = sentence.split()
    # Lowercase all words (default_stopwords are lowercase too)
    words = [word.lower() for word in words]
    #words = sentence
    words = [word for word in words if len(word) > 1]
    # Remove numbers
    words = [word for word in words if not word.isnumeric()]
    # Remove punctuation
    words = [word for word in words if word.isalpha()]
    # Remove stopwords
    words = [word for word in words if word not in stopwords]
    # Porter
    words = [porter_stemmer.stem(word) for word in words]
    #fdist = nltk.FreqDist(words_lc)   
    return " ".join(words)

def clean_df(df,input_col_name,output_col_name):
    """
    Clean the dataframe
    :param df: dataframe to be cleaned
    :param input_col_name: intput column name that contained the body text
    :param output_col_name: output column name
    :return: dataframe that has been cleaned with new column "text_clean"
    """
    # split to sentences
    #sample1['tokens_p_t'] = sample1['tokens_p'].apply(lambda x: x.split("."))
    df[output_col_name] = df.apply(lambda row: nltk.sent_tokenize(row[input_col_name]), axis=1)
    print (df[output_col_name])
    # clean sentences
    df[output_col_name] = df.apply(lambda row: clean_sent(row[output_col_name]), axis=1)
    return df

#### BERT

In [91]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text,max_length=500
                                )

    # Report how long the input sequence is.
    #print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    
    
    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
            
    s_scores = start_scores.detach().numpy().flatten()
    e_scores = end_scores.detach().numpy().flatten()
    #print('score:'+(start_scores)+"; "+str(end_scores))
    #print('score:'+str(max(s_scores))+"; "+str(min(e_scores)))
    #print(str(tensor[torch.argmax(start_scores)]))
    #print('Answer: "' + answer + '"')
    #[answer,str(max(s_scores)),len(input_ids)]
    return answer

### Clean text

In [92]:
#clean_pdf_df = clean_pdf_df[:100]

In [109]:
from timeit import default_timer as timer
start = timer()

In [110]:
clean_pdf_df['text_cleaned'] = clean_pdf_df.apply(lambda row: clean_sent(row['text']), axis=1)

In [111]:
end = timer()
clean_time = end-start
clean_time

2766.680513938016

In [385]:
text = clean_pdf_df['text_cleaned']

### Paper selection by similarity

In [386]:
questions = ["Can the virus be transmitted asymptomatically or during the incubation period"]

In [387]:
questions_cleaned = [clean_sent(ques) for ques in questions]

In [388]:
start = timer()

In [389]:
def calc_simlarity_score(question_list, text_list, threshold=None, top=None):
    dic = {}
    tfidf = TfidfVectorizer()
    corpus_tfidf_matrix = tfidf.fit_transform(text_list)
    ques_tfidf_matrix = tfidf.transform(question_list)
    sim_matrix = cosine_similarity(corpus_tfidf_matrix, ques_tfidf_matrix)
    for ques_idx in range(sim_matrix.shape[1]):
        dic[ques_idx] = []
        if threshold != None:
            for paper_idx in range(sim_matrix.shape[0]):
                # threshold
                score = sim_matrix[paper_idx, ques_idx]
                if score >= threshold:
                    dic[ques_idx].append((paper_idx, score))
        elif top != None:
            top_paper_idx_list = sorted(range(len(sim_matrix[:, ques_idx])), key=lambda i: sim_matrix[:,0][i], reverse=True)[:top]
            dic[ques_idx] = [(top_idx, sim_matrix[top_idx, ques_idx]) for top_idx in top_paper_idx_list]
        else:
            print ("NO!!!!!!!!!!")
            
    return dic, sim_matrix

In [390]:
def retrieve_paper(df, dic):
    for ques_idx in dic:
        new_df = df.iloc[[item[0] for item in dic[ques_idx]], :]
        new_df['score'] = [item[1] for item in dic[ques_idx]]
        new_df['question'] = questions[ques_idx]
    return new_df

In [391]:
dic, sim_matrix = calc_simlarity_score(questions_cleaned, text, top=100)

In [392]:
dic

{0: [(58526, 0.45289112747936044),
  (35221, 0.45047275149465865),
  (17334, 0.44869550787456114),
  (18650, 0.44384239284923094),
  (14308, 0.42155129141854975),
  (7822, 0.4130088265960259),
  (55643, 0.40191284238958536),
  (5233, 0.3971283961214767),
  (51504, 0.3822938315561995),
  (56937, 0.3754049155566371),
  (49805, 0.3709377974487783),
  (37053, 0.3688407486815889),
  (24196, 0.3658564069748685),
  (40896, 0.3543083657461978),
  (23076, 0.3508476120671627),
  (3591, 0.34605550392399165),
  (17278, 0.3442088296752532),
  (49349, 0.3429517766440414),
  (1017, 0.34172596298577324),
  (37828, 0.33939425359022757),
  (35537, 0.3339563602320891),
  (15655, 0.3334758463405723),
  (55981, 0.33091069302603515),
  (23834, 0.3279271746419207),
  (10740, 0.32084541695725105),
  (58457, 0.31998343498757315),
  (1172, 0.3155069427788312),
  (44301, 0.3046812052557716),
  (57041, 0.2985716798813143),
  (25666, 0.29799089050909444),
  (6426, 0.2978082065255916),
  (23345, 0.2928100880919165)

In [445]:
relevant_paper = retrieve_paper(clean_pdf_df, dic)
relevant_paper = relevant_paper.reset_index(drop=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [446]:
covid19_index_list = []
for i, row in relevant_paper.iterrows():
    text = row['text'].lower()
    has_keyword = False
    for keyword in keyword_list:
        if keyword in text:
            has_keyword = True
            break
    if has_keyword == True:
        covid19_index_list.append(i)

In [447]:
relevant_paper = relevant_paper.iloc[covid19_index_list, :]

In [448]:
relevant_paper = relevant_paper.reset_index(drop=False)

In [449]:
len(relevant_paper)

68

In [450]:
# The preprocessing needs to be done before cleaning.
relevant_paper['title_x'] = relevant_paper['title_x'].fillna(relevant_paper['title_y'])
relevant_paper['authors_x'] = relevant_paper['authors_x'].fillna(relevant_paper['authors_y'])
relevant_paper['abstract_x'] = relevant_paper['abstract_x'].fillna(relevant_paper['abstract_y'])

In [451]:
relevant_paper = relevant_paper.drop(['sha', 'title_y', 'authors_y', 'abstract_y'], axis=1)

In [452]:
relevant_paper = relevant_paper.rename(columns={'title_x': 'title', 'authors_x': 'authors', 'abstract_x': 'abstract'})

### Create target table 

#### General columns

In [577]:
relevant_paper.columns

Index(['index', 'paper_id', 'title', 'authors', 'affiliations', 'abstract',
       'text', 'bibliography', 'raw_authors', 'raw_bibliography', 'doi',
       'publish_time', 'journal', 'text_cleaned', 'score', 'question'],
      dtype='object')

In [587]:
target_table = pd.DataFrame(columns = ['Date', 'Study', 'Study Link', 'Journal', 'Study Type', 'Sample Size', 'Age', 
                                       'Sample Obtained', 'Asymptomatic Transmission', 'Characteristic Related to Question 2',
                                       'Excerpt', 'Added On'])

In [588]:
target_table.head()

,Date,Study,Study Link,Journal,Study Type,Sample Size,Age,Sample Obtained,Asymptomatic Transmission,Characteristic Related to Question 2,Excerpt,Added On


In [589]:
target_table[['Date', 'Study', 'Journal']] = relevant_paper[['publish_time', 'title', 'journal']]

In [590]:
target_table['Study Link'] = "https://doi.org/" + relevant_paper['doi']

#### Specific columns

In [586]:
question = "What's the range of the age?"
cnt = 0
has_excerpt_list = []
age_string_list = []
age_ans_list = []
for i, row in relevant_paper.iterrows():
    #print (i)
    sent_list = row['text'].split('. ')
    ans_list = [sent for sent in sent_list if ('asymptom' in sent) & ('transmis' in sent) & ('%' in sent)]
    age_list = [sent for sent in sent_list if (' age ' in sent.lower()) or (' aged' in sent.lower())]
    age_string = '; \n'.join([age for age in age_list])
    if age_string != '':
        age_ans = answer_question(question, age_string)
    else:
        age_ans = ''
    age_string_list.append(age_string)
    age_ans_list.append(age_ans)
    ans_string = '; \n'.join([ans for ans in ans_list])
    if ans_list:
        has_excerpt_list.append((i, ans_string))
        cnt += 1
print(cnt)

11


In [591]:
target_table['Age string'] = age_string_list
target_table['Age'] = age_ans_list
idx_list = [item[0] for item in has_excerpt_list]
target_table = target_table.iloc[idx_list, :]
target_table['Excerpt'] = [item[1] for item in has_excerpt_list]

In [592]:
pattern = '\d{2}(?:\.\d{1,2})?%'
re_search = re.compile(pattern)
re_search.findall(temp)

target_table['Asymptomatic Transmission'] = target_table['Excerpt'].map(lambda x: re_search.findall(x))

In [593]:
target_table

,Date,Study,Study Link,Journal,Study Type,Sample Size,Age,Sample Obtained,Asymptomatic Transmission,Characteristic Related to Question 2,Excerpt,Added On,Age string
5,2020-04-18,Journal Pre-proof The relative transmissibilit...,https://doi.org/10.1016/j.ijid.2020.04.034,Int J Infect Dis,NaN,NaN,,NaN,[41.6%],NaN,28 29 Keywords: COVID-19; asymptomatic cases; ...,NaN,
6,2020-05-11,Journal Pre-proof The time scale of asymptomat...,https://doi.org/10.1016/j.epidem.2020.100392,Epidemics,NaN,NaN,,NaN,"[86%, 79%]",NaN,Given that 86% of the cases were undocumented ...,NaN,
7,2019-02-28,Asymptomatic Middle East Respiratory Syndrome ...,https://doi.org/10.1016/j.tmaid.2018.12.003,Travel Medicine and Infectious Disease,NaN,NaN,,NaN,[91%],NaN,"In a study from Abu Dhabi, of 34 casepatients,...",NaN,
10,2020-04-14,Public Health Policy: COVID-19 Epidemic and SE...,NaN,NaN,NaN,NaN,between 20 and 50 years old,NaN,[40%],NaN,"From this figure, we can conclude that the asy...",NaN,[11] conclude that the contact numbers for pop...
22,2020-05-11,The contribution of asymptomatic SARS-CoV-2 in...,https://doi.org/10.1101/2020.05.07.20093849,NaN,NaN,NaN,45 - 75,NaN,"[53%, 56%, 83%, 69%, 74%, 69%, 85%, 56%, 76%, ...",NaN,We found that because systematic testing irres...,NaN,"At the start of quarantine there were 3,711 in..."
23,2020-04-29,The role of asymptomatic SARS-CoV-2 infections...,https://doi.org/10.1101/2020.04.25.20079103,NaN,NaN,NaN,13 months and 10 years,NaN,"[60%, 90%, 47%, 95%, 58%, 57%, 60%]",NaN,"In the studies included in this review, three ...",NaN,5 The ages were reported for six of the asympt...
27,2020-03-08,Transmission of corona virus disease 2019 duri...,https://doi.org/10.1101/2020.03.06.20031955,NaN,NaN,NaN,19 to 73,NaN,[66.2%],NaN,"Particularly, 66.2% of the secondary cases wer...",NaN,We defined the back-projection equation as fol...
30,2020-04-07,Comparison of transmissibility of coronavirus ...,https://doi.org/10.1101/2020.04.02.20050740,NaN,NaN,NaN,,NaN,"[95%, 95%, 95%]",NaN,"As a result, there is no evidence in the data ...",NaN,
43,2020-05-04,Journal Pre-proof Limited effectiveness of sys...,https://doi.org/10.1016/j.medmal.2020.04.020,Med Mal Infect,NaN,NaN,,NaN,[44%],NaN,"Second, viral transmission during the incubati...",NaN,
58,2020-04-22,Predicting the impact of asymptomatic transmis...,https://doi.org/10.1101/2020.04.16.20068387,NaN,NaN,NaN,,NaN,"[85%, 55%, 25%, 25%]",NaN,"To complicate the containment of the disease, ...",NaN,


In [594]:
question = "What is the sample size?"

In [601]:
temp_list = target_table['Excerpt'].to_list()
sample_size_list = []
for temp in temp_list:
    #print (temp)
    ans = answer_question(question, temp)
    if not ans.isdigit():
        ans = ''
    sample_size_list.append(ans)
    #print ("Answer: " + answer_question(question, temp))

In [603]:
target_table['Sample Size'] = sample_size_list